# Pond5 Crawler

In [ ]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import random
import time
import re
import os
import base64

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
import wget

## Import breeds from txt file

In [ ]:
final_breeds = []

path = os.getcwd()
my_file = open(os.path.join(path ,"Breed_pond5.txt"), "r")
content = my_file.read()
content_list = content.split("\n")
my_file.close()

for breed in content_list:
        word = breed.replace("_", " ").replace("-", " ")
        word= word[:1].upper() + word[1:]
        final_breeds.append(word)
        
print(final_breeds)

## Selenium driver setup

In [ ]:
chrome_options = webdriver.ChromeOptions()
s = Service('C:/Users/Asus/Documents/WebDriver/chromedriver.exe')
driver = webdriver.Chrome(service=s)
driver.get('https://www.pond5.com/')
time.sleep(5)

In [ ]:
main_URL = "https://www.pond5.com/"
page_url = "&pp="

## Pond5 Crawling/Scraping  Functions

In [ ]:
def create_folder(query):
    if not os.path.exists(os.getcwd() + '/' + query):
        path = os.getcwd()
        path = os.path.join(path, query)
        os.mkdir(path)
        print(query + " Folder Created")
    else:
        print(query + " Folder Already Exist")
        
def save_images(images, query):
    counter = 0
    path = os.getcwd()
    path = os.path.join(path, query)
    for img in images:
        save_as = os.path.join(path ,query + str(counter) +'.jpg')
        try:
            wget.download(img, save_as)
        except Exception as e:
            print("Could not download file %s"%(img))
            print(e)
            continue
        counter += 1
    
    print("%s Images Saved! | Total of: %s Images" %(query,counter))
    
def save_single_images(img, query):
    counter = 0
    path = os.getcwd()
    path = os.path.join(path, query)
    save_as = os.path.join(path ,query + str(counter) +'.jpg')
    print(img)
    wget.download(img, save_as)
    
    print("%s Images Saved!" %(query))

In [ ]:
def scroll_down(driver):
    img_urls = []
    try:
        # Get scroll height.
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            time.sleep(random.randint(1,3))
            page_source = driver.page_source
            soup = bs(page_source, 'html.parser')
            img_div = soup.find('div',{'class':'SearchResultsV3-mosaic js-searchResultsList'})
            links = img_div.find_all('img')
            for link in links:
                try:
                    img_urls.append(link['src'])
                except:
                    img_urls.append(link['data-src'])
            
            time.sleep(random.randint(2,4))

            #Remove Duplicates Of Img URLs
            img_urls = list( dict.fromkeys(img_urls))
            # Scroll down to the bottom.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
            # Wait to load the page.
            time.sleep(random.randint(3,6))
            # Calculate new scroll height and compare with last scroll height.
            new_height = driver.execute_script("return document.body.scrollHeight/2")
            if new_height == last_height:
                print("End Of Page..")
                break
            last_height = new_height
            
    except ElementNotInteractableException:
        raise Exception("Exception -> end of page")
        
    return img_urls

## Use The Automation For Each Breed

#### In this automation we used Selenium and BeautifulSoup combined

In [ ]:
wait = WebDriverWait(driver, 10)

for breed in final_breeds:
    
    print("Search images for: " + breed)
    img_urls = []
    img_download_links = []
    query = breed
    search_URL = f"https://www.pond5.com/search?kw={breed}&media=photo"
    page = driver.get(search_URL + page_url + "1") 

    time.sleep(random.randint(3,6))
    page_source = driver.page_source
    soup = bs(page_source, 'html.parser')
    total_pages = soup.find('div',{'class':'Arrange-sizeFit Arrange-right js-paginationCount'})
    print("Total pages to run: %s"%(total_pages.get_text()))
    
    num_page = int(total_pages.get_text())
    count=0
    for i in range(1, num_page+1):
        img_urls = scroll_down(driver)
        time.sleep(random.randint(2,4))
        for link in img_urls:
            try:
                img_download_links.append(link)
            except:
                continue        
        print("Total Images: %s"%(len(img_download_links)))
        if(len(img_download_links) > 5000):
            break
        page = driver.get(search_URL + page_url + str(i+1))
    
    folder_name = query + "Pond5"
    #create path if dosent exsist
    create_folder(folder_name)
    
    #save the images
    save_images(img_download_links,folder_name)
    print("Donwload for %s finished, Total of %s images!"%(breed,len(img_download_links)))

### In this project all the data came from scrapping and the crawling is used for educational purpose only.